<a href="https://colab.research.google.com/github/VictorUceda/TrackNet-Padel/blob/main/cut_point.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/VictorUceda/TrackNet-Padel

remote: Total 485 (delta 12), reused 42 (delta 11), pack-reused 442
Receiving objects: 100% (485/485), 459.21 MiB | 25.35 MiB/s, done.
Resolving deltas: 100% (230/230), done.
Checking out files: 100% (97/97), done.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

video_path = '/content/gdrive/MyDrive/PadelMatchs/WPTDanish_Octavos_PD_0_21.mp4'
points_path = video_path[:-4]+'.points.csv'
ball_path = video_path[:-4]+'.ballpos'
res = (1920,1080)

Mounted at /content/gdrive


In [3]:
%cd TrackNet-Padel

/content/TrackNet-Padel


In [ ]:
import pandas as pd

df_segments = pd.read_csv(points_path, sep=';')
i = 0
for index, row in df_segments.iterrows():
  name = video_path[:-4]+'_'+str(i)+'.mp4'
  ini = (int(row['ini']/25/60), (row['ini']%(25*60))/25)
  dur = (int((row['fin']-row['ini'])/25/60), ((row['fin']-row['ini'])%(25*60))/25)
  print('00:'+str(ini[0])+':'+str(ini[1]), '00:'+str(dur[0])+':'+str(dur[1]), name)
  !ffmpeg -y -ss {'00:'+str(ini[0])+':'+str(ini[1])} -i {video_path} -c copy -t {'00:'+str(dur[0])+':'+str(dur[1])} {name}
  i += 1

In [ ]:
!python cut_fragments.py --label_video_path={video_path} --output_segments_path={'/content/gdrive/MyDrive/PadelMatchs/'}

In [4]:
import pandas as pd
import pickle


with open(ball_path, "rb") as fp:  
  ball_pos = pickle.load(fp)

df_segments = pd.read_csv(points_path, sep=';')
j = 0
for index, row in df_segments.iterrows():
  name = video_path[:-4]+'_'+str(j)+'.csv'
  csv_str = 'Fram,Ball,x,y\n'
  for i in range(row['ini'], row['fin']):
    if i in ball_pos:
      csv_str +=  str(i-row['ini'])+",1,"+str(ball_pos[i][0][0]/res[0])+","+str(ball_pos[i][0][1]/res[1])+'\n'
      #print( str(i-row['ini'])+str(ball_pos[i][0]))
    else:
      csv_str +=  str(i-row['ini'])+",0,-1.000,-1.000\n"
  with open(name, "w") as fp:
    fp.write(csv_str)
  j += 1
